In [3]:
pip install lxml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 41.1 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
import requests
from lxml import html


In [6]:
# 스크레이핑 대상 URL
target_url = "https://huggingface.co/papers/week/2025-W33"

# 결과를 저장할 파일 이름
output_filename = "huggingface_top_2_papers_2025-W33.txt"

# 스크레이핑할 논문 개수
num_papers_to_scrape = 2

# 기본 URL (상대 경로를 절대 경로로 만들기 위해 사용)
base_url = "https://huggingface.co"



In [7]:
print(f"'{target_url}'에서 논문 목록을 가져옵니다...")
response = requests.get(target_url, timeout=10)
response.raise_for_status() # 요청 실패 시 예외 발생

print("페이지를 성공적으로 가져왔습니다.")
# print(response.text) # 전체 HTML을 확인하고 싶을 경우 주석 해제

'https://huggingface.co/papers/week/2025-W33'에서 논문 목록을 가져옵니다...
페이지를 성공적으로 가져왔습니다.


In [8]:
response


<Response [200]>

In [10]:
tree = html.fromstring(response.content)
tree

<Element html at 0x116c477a0>

In [11]:
# XPath를 사용하여 상위 2개의 논문 article 목록을 가져옴
paper_articles = tree.xpath("/html/body/div[1]/main/div[2]/section/div[2]/article")[:num_papers_to_scrape]

if not paper_articles:
    print("논문 정보를 찾을 수 없습니다. XPath나 페이지 구조를 확인해주세요.")
else:
    print(f"총 {len(paper_articles)}개의 논문 항목을 찾았습니다.")
    

총 2개의 논문 항목을 찾았습니다.


In [12]:
paper_articles

[<Element article at 0x116c8adf0>, <Element article at 0x116c8abc0>]

In [14]:
if paper_articles:
    print("--- 첫 번째 article 요소의 HTML 내용 ---")
    # tostring은 바이트(bytes)를 반환하므로 .decode()로 문자열로 변환해줍니다.
    print(html.tostring(paper_articles[0], pretty_print=True).decode('utf-8'))

    if len(paper_articles) > 1:
        print("\n--- 두 번째 article 요소의 HTML 내용 ---")
        print(html.tostring(paper_articles[1], pretty_print=True).decode('utf-8'))

--- 첫 번째 article 요소의 HTML 내용 ---
<article class="relative flex flex-col overflow-hidden rounded-xl border"><a href="/papers/2508.06471" class="shadow-alternate-sm peer relative block h-56 w-full cursor-pointer overflow-hidden rounded-xl bg-white md:h-64"><img src="https://cdn-thumbnails.huggingface.co/social-thumbnails/papers/2508.06471.png" loading="lazy" decoding="async" alt="" class="h-full w-full object-cover object-top opacity-80 dark:opacity-70 dark:invert"></a>

	<div class="shadow-xs pointer-events-none absolute right-2 top-56 -mt-8 flex h-6 items-center gap-1 self-end whitespace-nowrap rounded-md border bg-white px-2 text-xs leading-none text-gray-700 dark:bg-gray-900 dark:text-gray-400 sm:text-sm md:top-64">Submitted by
		<img alt="" loading="lazy" class="size-2.5 rounded-full  flex-none" src="/avatars/76c3b70e312f25e1e610473475553c5c.svg" crossorigin="anonymous">
			xianbao</div>

	

	<div class="from-gray-50-to-white bg-linear-to-b -mt-2 flex px-6 pb-6 pt-8"><div class="fle

In [18]:
print("파일에 저장을 시작합니다...")

with open(output_filename, "w", encoding="utf-8") as f:
    f.write(f"Hugging Face 주간 인기 논문 Top 2 (2025-W33)\n")
    f.write(f"출처: {target_url}\n\n")
    
    for i, article in enumerate(paper_articles):
        try:
            # article 태그 내부의 h3 태그를 먼저 찾음
            h3_element = article.xpath(".//h3")[0]

            # h3 태그 내부에서 링크(a 태그)와 제목(a 태그의 텍스트)을 찾음
            link_list = h3_element.xpath("./a/@href")
            title_list = h3_element.xpath("./a/text()")

            if not link_list or not title_list:
                raise ValueError("h3 태그 안에서 링크나 제목을 찾을 수 없습니다.")

            relative_link = link_list[0]
            # 제목이 여러 조각으로 나뉘어 있을 수 있으므로 합쳐줌
            title = "".join(title_list).strip()
            paper_url = f"{base_url}{relative_link}"
            
            print(f"\n({i+1}/{len(paper_articles)}) '{title}' 처리 중...")
            print(f"   > 상세 페이지로 이동: {paper_url}")

            # 파일에 제목과 링크 쓰기
            f.write(f"--- {i+1}. {title} ---\n")
            f.write(f"Link: {paper_url}\n\n")

            # 논문 상세 페이지로 접속
            paper_response = requests.get(paper_url, timeout=10)
            paper_response.raise_for_status()
            paper_tree = html.fromstring(paper_response.content)

            # [수정됨] 사용자가 제공한 구체적인 XPath로 초록(Abstract) 추출
            abstract_elements = paper_tree.xpath('/html/body/div/main/div/section[1]/div/div[2]/div/p')
            
            if abstract_elements:
                # .text_content()는 p 태그 내부의 모든 텍스트(a 태그 안의 텍스트 포함)를 가져옴
                abstract = abstract_elements[0].text_content().strip()
                print("   > 초록을 찾았습니다.")
                f.write("## Abstract\n")
                f.write(abstract + "\n\n")
            else:
                print("   > 초록을 찾을 수 없습니다.")
                f.write("초록을 찾을 수 없습니다.\n\n")

        except Exception as e:
            error_msg = f"항목 처리 중 오류 발생: {e}"
            print(f"   > 오류: {error_msg}")
            f.write(f"--- {i+1}. 항목 처리 실패 ---\n{error_msg}\n\n")

print(f"\n완료! 모든 정보가 '{output_filename}' 파일에 저장되었습니다.")

파일에 저장을 시작합니다...

(1/2) 'GLM-4.5: Agentic, Reasoning, and Coding (ARC) Foundation Models' 처리 중...
   > 상세 페이지로 이동: https://huggingface.co/papers/2508.06471
   > 초록을 찾았습니다.

(2/2) 'We-Math 2.0: A Versatile MathBook System for Incentivizing Visual
  Mathematical Reasoning' 처리 중...
   > 상세 페이지로 이동: https://huggingface.co/papers/2508.10433
   > 초록을 찾았습니다.

완료! 모든 정보가 'huggingface_top_2_papers_2025-W33.txt' 파일에 저장되었습니다.


# 최종 코드 진행중


In [ ]:
import requests
from lxml import html


In [19]:
# 스크레이핑 대상 URL
target_url = "https://huggingface.co/papers/week/2025-W33"

# 결과를 저장할 파일 이름
output_filename = "huggingface_top_10_papers_2025-W33.txt"

# 스크레이핑할 논문 개수
num_papers_to_scrape = 10

# 기본 URL (상대 경로를 절대 경로로 만들기 위해 사용)
base_url = "https://huggingface.co"

In [20]:
print(f"'{target_url}'에서 논문 목록을 가져옵니다...")
response = requests.get(target_url, timeout=10)
response.raise_for_status() # 요청 실패 시 예외 발생

print("페이지를 성공적으로 가져왔습니다.")
# print(response.text) # 전체 HTML을 확인하고 싶을 경우 주석 해제

'https://huggingface.co/papers/week/2025-W33'에서 논문 목록을 가져옵니다...
페이지를 성공적으로 가져왔습니다.


In [21]:
# HTML 파싱
tree = html.fromstring(response.content)

# XPath를 사용하여 상위 10개의 논문 article 목록을 가져옴
paper_articles = tree.xpath("/html/body/div[1]/main/div[2]/section/div[2]/article")[:num_papers_to_scrape]

if not paper_articles:
    print("논문 정보를 찾을 수 없습니다. XPath나 페이지 구조를 확인해주세요.")
else:
    print(f"총 {len(paper_articles)}개의 논문 항목을 찾았습니다.")
    print(paper_articles) # 객체 리스트 확인


총 10개의 논문 항목을 찾았습니다.
[<Element article at 0x116d07e30>, <Element article at 0x116d19a90>, <Element article at 0x116d19ae0>, <Element article at 0x116d199f0>, <Element article at 0x116d18960>, <Element article at 0x116d18b40>, <Element article at 0x116d19bd0>, <Element article at 0x116d19b30>, <Element article at 0x116d19c70>, <Element article at 0x116d19cc0>]


In [22]:
# paper_articles 리스트가 비어있지 않은 경우에만 실행
if paper_articles:
    print("--- 첫 번째 article 요소의 HTML 내용 ---")
    # tostring은 바이트(bytes)를 반환하므로 .decode()로 문자열로 변환해줍니다.
    print(html.tostring(paper_articles[0], pretty_print=True).decode('utf-8'))

    if len(paper_articles) > 1:
        print("\n--- 두 번째 article 요소의 HTML 내용 ---")
        print(html.tostring(paper_articles[1], pretty_print=True).decode('utf-8'))

--- 첫 번째 article 요소의 HTML 내용 ---
<article class="relative flex flex-col overflow-hidden rounded-xl border"><a href="/papers/2508.06471" class="shadow-alternate-sm peer relative block h-56 w-full cursor-pointer overflow-hidden rounded-xl bg-white md:h-64"><img src="https://cdn-thumbnails.huggingface.co/social-thumbnails/papers/2508.06471.png" loading="lazy" decoding="async" alt="" class="h-full w-full object-cover object-top opacity-80 dark:opacity-70 dark:invert"></a>

	<div class="shadow-xs pointer-events-none absolute right-2 top-56 -mt-8 flex h-6 items-center gap-1 self-end whitespace-nowrap rounded-md border bg-white px-2 text-xs leading-none text-gray-700 dark:bg-gray-900 dark:text-gray-400 sm:text-sm md:top-64">Submitted by
		<img alt="" loading="lazy" class="size-2.5 rounded-full  flex-none" src="/avatars/76c3b70e312f25e1e610473475553c5c.svg" crossorigin="anonymous">
			xianbao</div>

	

	<div class="from-gray-50-to-white bg-linear-to-b -mt-2 flex px-6 pb-6 pt-8"><div class="fle

In [23]:
print("파일에 저장을 시작합니다...")

with open(output_filename, "w", encoding="utf-8") as f:
    f.write(f"Hugging Face 주간 인기 논문 Top 10 (2025-W33)\n")
    f.write(f"출처: {target_url}\n\n")
    
    for i, article in enumerate(paper_articles):
        try:
            # article 태그 내부의 h3 태그를 먼저 찾음
            h3_element = article.xpath(".//h3")[0]

            # h3 태그 내부에서 링크(a 태그)와 제목(a 태그의 텍스트)을 찾음
            link_list = h3_element.xpath("./a/@href")
            title_list = h3_element.xpath("./a/text()")

            if not link_list or not title_list:
                raise ValueError("h3 태그 안에서 링크나 제목을 찾을 수 없습니다.")

            relative_link = link_list[0]
            # 제목이 여러 조각으로 나뉘어 있을 수 있으므로 합쳐줌
            title = "".join(title_list).strip()
            paper_url = f"{base_url}{relative_link}"
            
            print(f"\n({i+1}/{len(paper_articles)}) '{title}' 처리 중...")
            print(f"   > 상세 페이지로 이동: {paper_url}")

            # 파일에 제목과 링크 쓰기
            f.write(f"--- {i+1}. {title} ---\n")
            f.write(f"Link: {paper_url}\n\n")

            # 논문 상세 페이지로 접속
            paper_response = requests.get(paper_url, timeout=10)
            paper_response.raise_for_status()
            paper_tree = html.fromstring(paper_response.content)

            # 사용자가 제공한 구체적인 XPath로 초록(Abstract) 추출
            abstract_elements = paper_tree.xpath('/html/body/div/main/div/section[1]/div/div[2]/div/p')
            
            if abstract_elements:
                # .text_content()는 p 태그 내부의 모든 텍스트(a 태그 안의 텍스트 포함)를 가져옴
                abstract = abstract_elements[0].text_content().strip()
                print("   > 초록을 찾았습니다.")
                f.write("## Abstract\n")
                f.write(abstract + "\n\n")
            else:
                print("   > 초록을 찾을 수 없습니다.")
                f.write("초록을 찾을 수 없습니다.\n\n")

        except Exception as e:
            error_msg = f"항목 처리 중 오류 발생: {e}"
            print(f"   > 오류: {error_msg}")
            f.write(f"--- {i+1}. 항목 처리 실패 ---\n{error_msg}\n\n")

print(f"\n완료! 모든 정보가 '{output_filename}' 파일에 저장되었습니다.")

파일에 저장을 시작합니다...

(1/10) 'GLM-4.5: Agentic, Reasoning, and Coding (ARC) Foundation Models' 처리 중...
   > 상세 페이지로 이동: https://huggingface.co/papers/2508.06471
   > 초록을 찾았습니다.

(2/10) 'We-Math 2.0: A Versatile MathBook System for Incentivizing Visual
  Mathematical Reasoning' 처리 중...
   > 상세 페이지로 이동: https://huggingface.co/papers/2508.10433
   > 초록을 찾았습니다.

(3/10) 'NextStep-1: Toward Autoregressive Image Generation with Continuous
  Tokens at Scale' 처리 중...
   > 상세 페이지로 이동: https://huggingface.co/papers/2508.10711
   > 초록을 찾았습니다.

(4/10) 'WebWatcher: Breaking New Frontier of Vision-Language Deep Research Agent' 처리 중...
   > 상세 페이지로 이동: https://huggingface.co/papers/2508.05748
   > 초록을 찾았습니다.

(5/10) 'ReasonRank: Empowering Passage Ranking with Strong Reasoning Ability' 처리 중...
   > 상세 페이지로 이동: https://huggingface.co/papers/2508.07050
   > 초록을 찾았습니다.

(6/10) 'WideSearch: Benchmarking Agentic Broad Info-Seeking' 처리 중...
   > 상세 페이지로 이동: https://huggingface.co/papers/2508.07999
   > 초록을 찾았습니

LLM 번역 추가

In [24]:
import os
import requests
from lxml import html
from openai import OpenAI
from dotenv import load_dotenv

In [25]:
# .env 파일에서 환경 변수를 불러옵니다.
load_dotenv()

# .env 파일에 OPENAI_API_KEY="your_key_here" 형식으로 키를 저장하세요.
API_KEY = os.getenv("OPENAI_API_KEY")

# API 키가 설정되지 않았을 경우 경고 메시지를 출력합니다.
if not API_KEY:
    print("경고: OPENAI_API_KEY 환경 변수를 찾을 수 없습니다.")
    print(".env 파일이 존재하고, 그 안에 OPENAI_API_KEY가 올바르게 설정되었는지 확인해주세요.")
    # 이 셀 이후의 코드에서 오류가 발생할 수 있습니다.
else:
    print("OpenAI API 키를 성공적으로 불러왔습니다.")
    client = OpenAI(api_key=API_KEY)

def translate_text_with_openai(text):
    """
    OpenAI API를 사용하여 주어진 텍스트를 한글로 번역합니다.
    """
    if not text or not text.strip() or not API_KEY:
        return "번역할 텍스트가 없거나 API 키가 설정되지 않았습니다."
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a helpful assistant that translates English text to Korean."},
                {"role": "user", "content": f"Translate the following English text to Korean:\n\n{text}"}
            ],
            temperature=0.3,
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"번역 중 오류 발생: {e}"

OpenAI API 키를 성공적으로 불러왔습니다.


In [26]:
# 스크레이핑 대상 URL
target_url = "https://huggingface.co/papers/week/2025-W33"

# 결과를 저장할 파일 이름
output_filename = "huggingface_top_10_papers_2025-W33_translated.txt"

# 스크레이핑할 논문 개수
num_papers_to_scrape = 10

# 기본 URL (상대 경로를 절대 경로로 만들기 위해 사용)
base_url = "https://huggingface.co"


In [27]:
print(f"'{target_url}'에서 논문 목록을 가져옵니다...")
response = requests.get(target_url, timeout=10)
response.raise_for_status() # 요청 실패 시 예외 발생

print("페이지를 성공적으로 가져왔습니다.")

'https://huggingface.co/papers/week/2025-W33'에서 논문 목록을 가져옵니다...
페이지를 성공적으로 가져왔습니다.


In [28]:
# HTML 파싱
tree = html.fromstring(response.content)

# XPath를 사용하여 상위 10개의 논문 article 목록을 가져옴
paper_articles = tree.xpath("/html/body/div[1]/main/div[2]/section/div[2]/article")[:num_papers_to_scrape]

if not paper_articles:
    print("논문 정보를 찾을 수 없습니다. XPath나 페이지 구조를 확인해주세요.")
else:
    print(f"총 {len(paper_articles)}개의 논문 항목을 찾았습니다.")

총 10개의 논문 항목을 찾았습니다.


In [29]:
print("파일에 저장을 시작합니다...")

with open(output_filename, "w", encoding="utf-8") as f:
    f.write(f"Hugging Face 주간 인기 논문 Top 10 (2025-W33) - 번역본\n")
    f.write(f"출처: {target_url}\n\n")
    
    for i, article in enumerate(paper_articles):
        try:
            h3_element = article.xpath(".//h3")[0]
            link_list = h3_element.xpath("./a/@href")
            title_list = h3_element.xpath("./a/text()")

            if not link_list or not title_list:
                raise ValueError("h3 태그 안에서 링크나 제목을 찾을 수 없습니다.")

            relative_link = link_list[0]
            title = "".join(title_list).strip()
            paper_url = f"{base_url}{relative_link}"
            
            print(f"\n({i+1}/{len(paper_articles)}) '{title}' 처리 중...")
            print(f"   > 상세 페이지로 이동: {paper_url}")

            f.write(f"--- {i+1}. {title} ---\n")
            f.write(f"Link: {paper_url}\n\n")

            paper_response = requests.get(paper_url, timeout=10)
            paper_response.raise_for_status()
            paper_tree = html.fromstring(paper_response.content)

            abstract_elements = paper_tree.xpath('/html/body/div/main/div/section[1]/div/div[2]/div/p')
            
            if abstract_elements:
                abstract = abstract_elements[0].text_content().strip()
                print("   > 초록을 찾았습니다. 번역을 시작합니다...")
                
                # 원문 초록 저장
                f.write("## Abstract (Original)\n")
                f.write(abstract + "\n\n")
                
                # 번역 실행
                translated_abstract = translate_text_with_openai(abstract)
                print("   > 번역 완료.")
                
                # 번역된 초록 저장
                f.write("## 초록 (Korean)\n")
                f.write(translated_abstract + "\n\n")
            else:
                print("   > 초록을 찾을 수 없습니다.")
                f.write("초록을 찾을 수 없습니다.\n\n")

        except Exception as e:
            error_msg = f"항목 처리 중 오류 발생: {e}"
            print(f"   > 오류: {error_msg}")
            f.write(f"--- {i+1}. 항목 처리 실패 ---\n{error_msg}\n\n")

print(f"\n완료! 모든 정보가 '{output_filename}' 파일에 저장되었습니다.")

파일에 저장을 시작합니다...

(1/10) 'GLM-4.5: Agentic, Reasoning, and Coding (ARC) Foundation Models' 처리 중...
   > 상세 페이지로 이동: https://huggingface.co/papers/2508.06471
   > 초록을 찾았습니다. 번역을 시작합니다...
   > 번역 완료.

(2/10) 'We-Math 2.0: A Versatile MathBook System for Incentivizing Visual
  Mathematical Reasoning' 처리 중...
   > 상세 페이지로 이동: https://huggingface.co/papers/2508.10433
   > 초록을 찾았습니다. 번역을 시작합니다...
   > 번역 완료.

(3/10) 'NextStep-1: Toward Autoregressive Image Generation with Continuous
  Tokens at Scale' 처리 중...
   > 상세 페이지로 이동: https://huggingface.co/papers/2508.10711
   > 초록을 찾았습니다. 번역을 시작합니다...
   > 번역 완료.

(4/10) 'WebWatcher: Breaking New Frontier of Vision-Language Deep Research Agent' 처리 중...
   > 상세 페이지로 이동: https://huggingface.co/papers/2508.05748
   > 초록을 찾았습니다. 번역을 시작합니다...
   > 번역 완료.

(5/10) 'ReasonRank: Empowering Passage Ranking with Strong Reasoning Ability' 처리 중...
   > 상세 페이지로 이동: https://huggingface.co/papers/2508.07050
   > 초록을 찾았습니다. 번역을 시작합니다...
   > 번역 완료.

(6/10) 'WideSea